
## Projeto ICMC Júnior
### Objetivo: Prever se um funcionário vai sair da empresa (attrition).

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Leitura do arquivo csv
df = pd.read_csv('dados.csv')
# Verificação do tamanho da base de dados
df.shape

(1470, 35)

In [5]:
def calculate_lim_iqr(data_column):
    """Função responsável por calcular a amplitude interquartil de um Série (Coluna)"""
    Q1 = data_column.quantile(0.25) # 1° Quartil 
    Q3 = data_column.quantile(0.75) # 3° Quartil 
    
    
    IQR = Q3 - Q1 # amplitude interquartil
    
    limite_inferior = Q1 - (1.5 * IQR) # lim inferior do box-plot
    limite_superior = Q3 + (1.5 * IQR) # lim superior do box-plot
    
    return limite_inferior, limite_superior

def verify_outliers(column):
    # Calcula limite inferior e superior da Série fornecida (coluna)
    lim_inf, lim_sup = calculate_lim_iqr(df[column])
    
    # Verifica se possui um dado menor que o limite inferior ou se há um dado maior que o limite superior
    if df[column].min() < lim_inf or df[column].max() > lim_sup:
        return True
    return False

In [6]:
# Exibe os 5 primeiros funcionários da base de dados, apenas para contextualizar com a base
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1.0,2,Life Sciences,1.0,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8.0,1,Life Sciences,1.0,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2.0,2,Other,1.0,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,NaN,4,Life Sciences,NaN,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,NaN,591,Research & Development,2.0,1,Medical,NaN,7,...,4,80,1,6,3,3,2,2,2,2


In [7]:
# TRATAMENTO DE VALORES NULOS
df.isnull().sum() # Verifica quantos valores nulos tem por colunas, ou seja, por categoria.

Age                           0
Attrition                     0
BusinessTravel               70
DailyRate                     0
Department                    0
DistanceFromHome             47
Education                     0
EducationField                0
EmployeeCount               220
EmployeeNumber                0
EnvironmentSatisfaction      53
Gender                       67
HourlyRate                   32
JobInvolvement               61
JobLevel                      0
JobRole                       0
JobSatisfaction               0
MaritalStatus                 0
MonthlyIncome                 0
MonthlyRate                   0
NumCompaniesWorked            0
Over18                        0
OverTime                     51
PercentSalaryHike             0
PerformanceRating            46
RelationshipSatisfaction      0
StandardHours                 0
StockOptionLevel              0
TotalWorkingYears             0
TrainingTimesLastYear         0
WorkLifeBalance               0
YearsAtC

In [8]:
df['EmployeeCount']

0       1.0
1       1.0
2       1.0
3       NaN
4       NaN
       ... 
1465    1.0
1466    1.0
1467    1.0
1468    1.0
1469    1.0
Name: EmployeeCount, Length: 1470, dtype: float64

In [9]:
# Em 'EmployeeCount' todos os valores são como padrão igual a 1.0, sendo cada linha, um funcionário, com isso se corrige os NaN para 1.0
# no dataframe original
df['EmployeeCount'].fillna(value=1, inplace=True)

In [10]:
# Tratamento dos valores nulos da coluna: DistanceFromHome (Quantitativa continua)
# Calculando o IQR = Q3 - Q1

# Calcula o limite inferior e limite superior da coluna DistanceFromHome
lim_inf_dfh,lim_sup_dfh = calculate_lim_iqr(df['DistanceFromHome'])

print(f"Limite Inferior (Q1 - 1.5*IQR): {lim_inf_dfh}")
print(f"Limite Superior (Q3 + 1.5*IQR): {lim_sup_dfh}")

Limite Inferior (Q1 - 1.5*IQR): -16.0
Limite Superior (Q3 + 1.5*IQR): 32.0


In [11]:
# Resumo das descrições estatísticas da categoria DistanceFromHome
df['DistanceFromHome'].describe()

count    1423.000000
mean        9.188335
std         8.133707
min         1.000000
25%         2.000000
50%         7.000000
75%        14.000000
max        29.000000
Name: DistanceFromHome, dtype: float64

In [12]:
# O menor valor (min) é igual a 1.0 e o maior valor (max) é igual a 29.0, com isso podemos ver atráves disso, todos 
# os dados então dentro das margens, não tendo nenhum outliers. No entanto, devido ao desvio padrão ser mais de 80% da média, 
# indica que meus dados, não estão bem distribuidos. 

# Calculo a mediana
median_distances = df['DistanceFromHome'].median() 
# Preenche todos os valores NaN com a mediana e em seguida, reatribui a coluna nela mesma.
df['DistanceFromHome'] = df['DistanceFromHome'].fillna(median_distances)


In [13]:
print(df['EnvironmentSatisfaction'].value_counts(dropna=False))

EnvironmentSatisfaction
3.0    438
4.0    427
2.0    277
1.0    275
NaN     53
Name: count, dtype: int64


In [14]:
# Tratamento da coluna EnvironmentSatisfaction - Qualitativa ordinal
# Preenche os valores ausentes com os valores da moda, em seguida, reatribui os valores na coluna original do dataframe
# Obs: a mediana aqui teria o mesmo resultado da moda.

# Pega o primeiro (e único) valor da moda
moda_env_satisfaction = df['EnvironmentSatisfaction'].mode()[0]
df['EnvironmentSatisfaction'] = df['EnvironmentSatisfaction'].fillna(moda_env_satisfaction)

In [15]:
print(df['OverTime'].value_counts(dropna=False, normalize=True)) # Exibe a proporção de cada resposta na coluna OverTime, variando de 0 a 1.

OverTime
No     0.693197
Yes    0.272109
NaN    0.034694
Name: proportion, dtype: float64


In [16]:
# Tratamento de valores ausentes coluna - OverTime (Qualitativo nominal)
# Como podemos ver acima, possui uma quantidade esmagadora de funcionários que não realizam horas extras, quase 70% deles.

# Preenche os valores ausentes com a moda e reatribui os resultados na coluna respectiva
df['OverTime'] = df['OverTime'].fillna(df['OverTime'].mode()[0])


In [17]:
print(df['HourlyRate'].describe())

count    1438.000000
mean       65.907510
std        20.386193
min        30.000000
25%        48.000000
50%        66.000000
75%        84.000000
max       100.000000
Name: HourlyRate, dtype: float64


In [18]:
# Tratamento valores ausentes - HourlyRate (Quantitativo Contínua)

# Calcula o limite inferior e limite superior da coluna HourlyRate
lim_inf_hr, lim_sup_hr = calculate_lim_iqr(df['HourlyRate']) 
print(f"Limite inferior:{lim_inf_hr}, limite superior: {lim_sup_hr}")
print(f"Coeficiente de variação {df['HourlyRate'].std() / df['HourlyRate'].mean()}")
print(f"Média: { df['HourlyRate'].mean()} e Mediana: { df['HourlyRate'].median()}")

Limite inferior:-6.0, limite superior: 138.0
Coeficiente de variação 0.3093151741293963
Média: 65.90751043115438 e Mediana: 66.0


In [19]:
# Podemos ver atráves dos limites (inferior e superior) que os nossos dados estão dentro dessas margens, visto que
# o menor valor (min) é 30.0 e o maior (max) é 100.0, com isso não possui outliers. O coeficiente de variação é igual a 30%, dessa forma
# temos uma variação média entre os dados. Além disso, a média está bem proxima da mediana. Usa-se a média para representação dos dados ausentes

# Preenche os valores ausentes com a média e reatribui os resultados na coluna respectiva
df['HourlyRate'] = df['HourlyRate'].fillna(df['HourlyRate'].mean())

In [20]:
# Verifica se possui funcionários duplicados
df.duplicated().sum() 

np.int64(0)

In [21]:
df.describe()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1409.000000,1470.000000,...,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.118367,2.912925,1.0,1024.865306,2.727891,65.907510,2.732434,2.063946,...,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.011794,1.024165,0.0,602.024335,1.074222,20.162929,0.707171,1.106940,...,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,491.250000,2.000000,48.000000,2.000000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,1020.500000,3.000000,65.907510,3.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,13.000000,4.000000,1.0,1555.750000,4.000000,83.000000,3.000000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,5.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [22]:
# !adicionar depois as variávies que possuem valores ausentes ainda 
# ('BusinessTravel','Gender', 'JobInvolvement', 'PerformanceRating') -- Cols_qualitativas

# (Texto ou Números que são Rótulos/Categorias) 
cols_qualitativas = [
    'Attrition', 
    'Department', 
    'Education',
    'EducationField',
    'EnvironmentSatisfaction',
    'JobLevel', 
    'JobRole', 
    'JobSatisfaction',
    'MaritalStatus',
    'OverTime', 
    'RelationshipSatisfaction',
    'StockOptionLevel',
    'WorkLifeBalance'
]

# (Números que são Medidas/Contagens)
cols_quantitativas = [
    'Age',
    'DailyRate',
    'DistanceFromHome',
    'HourlyRate',
    'MonthlyIncome',
    'MonthlyRate',
    'NumCompaniesWorked',
    'PercentSalaryHike',
    'TotalWorkingYears',
    'TrainingTimesLastYear',
    'YearsAtCompany',
    'YearsInCurrentRole',
    'YearsSinceLastPromotion',
    'YearsWithCurrManager'
]

# Verificação se alguma das colunas quantitativa possui outliers 
print(10*'-','Colunas que podem conter outliers:', 10*'-')
for i in cols_quantitativas:
    if verify_outliers(i):
        print(f"{i}")
print(f"\n{10*'-'}Colunas que pode conter dados inconsistentes{10*'-'}")
for coluna in cols_qualitativas:
    print(f"\n===== Análise da Coluna: {coluna} =====")
    print(df[coluna].value_counts())
    print("---------------------------------")        
    

---------- Colunas que podem conter outliers: ----------
MonthlyIncome
NumCompaniesWorked
TotalWorkingYears
TrainingTimesLastYear
YearsAtCompany
YearsInCurrentRole
YearsSinceLastPromotion
YearsWithCurrManager

----------Colunas que pode conter dados inconsistentes----------

===== Análise da Coluna: Attrition =====
Attrition
No     1233
Yes     237
Name: count, dtype: int64
---------------------------------

===== Análise da Coluna: Department =====
Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64
---------------------------------

===== Análise da Coluna: Education =====
Education
3    572
4    398
2    282
1    170
5     48
Name: count, dtype: int64
---------------------------------

===== Análise da Coluna: EducationField =====
EducationField
Life Sciences       606
Medical             464
Marketing           159
Technical Degree    132
Other                82
Human Resources      27
Name: count, dtype: in

In [25]:
# Podemos que será necessário tratar os outliers de algumas colunas.
# Da saída acima é possível verificar também, que não possuimos valores inconsistes para erros de digitação, mas podemos verificar alguns
# valores que podem estar como algum valor do HourlyRate > DailyRate

print((df['HourlyRate'] > df['DailyRate']).sum()) # Cria uma Série booleana e soma todos os casos em que a condição é True
print((df['DailyRate'] > df['MonthlyRate']).sum()) # Cria uma Série booleana e soma todos os casos em que a condição é True

0
0


In [26]:
#  Tratamento de outliers